# Testing
### Zähldaten der Passagiere an der Hardbrücke
Datum: 04.01.2022

### Importiere die notwendigen Packages

In [1]:
#%pip install openpyxl geopandas altair fiona requests folium mplleaflet contextily seaborn datetime plotly

In [2]:
import pandas as pd
import pivottablejs
from pivottablejs import pivot_ui
import numpy as np
import altair as alt
import matplotlib.pyplot as plt
#from datetime import datetime
import datetime
import geopandas as gpd
import folium 
import time


Definiere Settings. Hier das Zahlenformat von Float-Werten (z.B. *'{:,.2f}'.format* mit Komma als Tausenderzeichen), 

In [3]:
pd.options.display.float_format = '{:.1f}'.format
pd.set_option('display.width', 100)
pd.set_option('display.max_columns', 15)

### Zeitvariabeln
Bestimme den aktuellst geladenen Monat. Hier ist es der Stand vor 2 Monaten. 
Bestimme noch weitere evt. sinnvolle Zeitvariabeln.

Zum Unterschied zwischen import datetime und from datedtime import datetime, siehe https://stackoverflow.com/questions/15707532/import-datetime-v-s-from-datetime-import-datetime

Zuerst die Zeitvariabeln als Strings

In [4]:
#today_date = datetime.date.today()
#date_time = datetime.datetime.strptime(date_time_string, '%Y-%m-%d %H:%M')
now = datetime.date.today()
date_today = now.strftime("%Y-%m-%d")
year_today = now.strftime("%Y")
month_today = now.strftime("%m")
day_today = now.strftime("%d")



Und hier noch die Zeitvariabeln als Integers:
- `aktuellesJahr`
- `aktuellerMonat`: Der gerade jetzt aktuelle Monat
- `selectedMonat`: Der aktuellste Monat in den Daten. In der Regel zwei Monate her.

In [5]:
#now = datetime.now() 
int_times = now.timetuple()

aktuellesJahr = int_times[0]
aktuellerMonat = int_times[1]
selectedMonat = int_times[1]-2

print(aktuellesJahr, 
      aktuellerMonat,
    'datenstand: ', selectedMonat,
     int_times)


2022 1 datenstand:  -1 time.struct_time(tm_year=2022, tm_mon=1, tm_mday=18, tm_hour=0, tm_min=0, tm_sec=0, tm_wday=1, tm_yday=18, tm_isdst=-1)


### Importiere einen Datensatz 

- Beachte dabei die Notation des Pfades...
- Definiere mal aktuell noch keine weiteren Parameter beim Import

**Dataset auf INTEG Datenkatalog**:  https://data.integ.stadt-zuerich.ch/dataset/vbz_frequenzen_hardbruecke/

**Dataset auf PROD Datenkatalog**:  https://data.stadt-zuerich.ch/dataset/vbz_frequenzen_hardbruecke/

In [6]:
#Die Datasets sind nur zum Testen auf INT-DWH-Dropzone. Wenn der Test vorbei ist, sind sie auf PROD. 
# Über den Status kann man einfach switchen

status = "prod";
print(status)

prod


In [7]:
# Filepath
if status == "prod":
    #fp = r"//szh.loc/ssz/applikationen/OGD_Dropzone/DWH/bev_monat_bestand_quartier_geschl_ag_herkunft_od3250/BEV325OD3250.csv"
    fp = r"https://data.stadt-zuerich.ch/dataset/vbz_frequenzen_hardbruecke/download/frequenzen_hardbruecke_2021.csv"

else:
    fp = r""


print(fp)

https://data.stadt-zuerich.ch/dataset/vbz_frequenzen_hardbruecke/download/frequenzen_hardbruecke_2021.csv


#### Parsing & Formatting Strings to Dataframes
- [**Cheat Sheet**](https://strftime.org/)
- Python documentation for **strptime**: [string **parse** time, Python 3](https://docs.python.org/3/library/datetime.html#datetime.datetime.strptime)
- Python documentation for strptime/**strftime**: [string **format** time,Python 3](https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior)



In [8]:
# Read the data
data2betested = pd.read_csv(
    fp, 
    sep=',',
    parse_dates=['Timestamp'],
    # KONVERTIERE DAS SAS DATUM IN EIN UNIXDATUM UND FORMATIERE ES
    #date_parser=lambda x: datetime.datetime.strptime(x, '%Y-%m-%dT%H:%M:%S'),
    low_memory=False    
    )

print('done')

done


In der Folge ein paar erste Tests:

 - 1) Zeige eine kurze Vorschau der importierten Daten
 - 2) Weise die Datentypen aus
 - 3) Zeige die Shape (Umfang) des Datensatzes an

In [9]:
data2betested

,In,Out,Timestamp,Name
0,1,1,2021-01-01 23:55:00,Ost-Nord total
1,2,0,2021-01-01 23:35:00,Ost-Nord total
2,0,1,2021-01-01 23:30:00,Ost-Nord total
3,3,6,2021-01-01 23:20:00,Ost-Nord total
4,0,2,2021-01-01 23:05:00,Ost-Nord total
...,...,...,...,...
725851,4,10,2022-01-01 00:20:00,West-VBZ total
725852,14,9,2022-01-01 00:15:00,West-VBZ total
725853,1,3,2022-01-01 00:10:00,West-VBZ total
725854,0,2,2022-01-01 00:05:00,West-VBZ total


In [10]:
data2betested.dtypes

In                    int64
Out                   int64
Timestamp    datetime64[ns]
Name                 object
dtype: object

Beschreibe einzelne Attribute

In [11]:
data2betested.describe()

,In,Out
count,725856.0,725856.0
mean,12.7,10.6
std,15.4,12.3
min,0.0,0.0
25%,3.0,3.0
50%,7.0,7.0
75%,17.0,14.0
max,422.0,445.0


Wie viele Nullwerte gibt es im Datensatz?

In [12]:
data2betested.isnull().sum()

In           0
Out          0
Timestamp    0
Name         0
dtype: int64

### Verwende das Datum als Index

While we did already parse the `datetime` column into the respective datetime type, it currently is just a regular column. 
**To enable quick and convenient queries and aggregations, we need to turn it into the index of the DataFrame**

In [13]:
data2betested = data2betested.set_index("Timestamp")
#data2betested = data2betested.drop(columns=["StichtagDatMM"])

In [14]:
data2betested.info()
data2betested.index.day.unique()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 725856 entries, 2021-01-01 23:55:00 to 2022-01-01 00:00:00
Data columns (total 3 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   In      725856 non-null  int64 
 1   Out     725856 non-null  int64 
 2   Name    725856 non-null  object
dtypes: int64(2), object(1)
memory usage: 22.2+ MB


Int64Index([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
            23, 24, 25, 26, 27, 28, 29, 30, 31],
           dtype='int64', name='Timestamp')

### Einfache Visualisierungen zur Plausi

Exploriere die Daten mit Pivottable.JS

In [15]:
#from pivottablejs import pivot_ui

#pivot_ui(data2betested)

### Zeitpunkte und Zeiträume abfragen

A particular powerful feature of the Pandas DataFrame is its indexing capability that also works using time-based entities, such as dates and times. We have already created the index above, so let's put it to use.

In [16]:
data2betested.loc["2021-06-30 23:05:00"]
#data2betested.loc["2021-12-04":"2021-12-06"]


,In,Out,Name
Timestamp,,,
2021-06-30 23:05:00,6,0,Ost-Nord total
2021-06-30 23:05:00,13,9,Ost-SBB total
2021-06-30 23:05:00,6,3,Ost-Süd total
2021-06-30 23:05:00,3,1,Ost-VBZ Total
2021-06-30 23:05:00,5,0,West-Nord total
2021-06-30 23:05:00,12,11,West-SBB total
2021-06-30 23:05:00,2,4,West-Süd total
2021-06-30 23:05:00,4,10,West-VBZ total


### Aggegiere Werte nach Zeitausschnitten mit RESAMPLE()

Mit den Funktionen zur Zeit kann einfach zwischen Stunden, Tagen, Monaten, etc. gewechselt und aggregiert werden.

Wenn ich nicht explizit groupby() für die Kategorien (hier die Zählorte) verwende, summiert resample() einfach nur die numerischen Werte, also alle Zählwerte, egal bei welchem Ort.


In [17]:

bsp_sum = data2betested.groupby('Name').resample("M").sum().head(13)
#Um die DataFrameGroup zurück zu setzen verwende ich reset_index()
bsp_sum  = bsp_sum.reset_index()
#Um die Zeitangabe wieder als Index zu verwenden muss ich nochmals set_index() anwenden
bsp_sum  = bsp_sum.set_index('Timestamp')
bsp_sum 

,Name,In,Out
Timestamp,,,
2021-01-31,Ost-Nord total,27859,43073
2021-02-28,Ost-Nord total,30064,42679
2021-03-31,Ost-Nord total,42508,57257
2021-04-30,Ost-Nord total,46931,61949
2021-05-31,Ost-Nord total,48509,65014
2021-06-30,Ost-Nord total,60170,80258
2021-07-31,Ost-Nord total,54166,77585
2021-08-31,Ost-Nord total,53978,75458
2021-09-30,Ost-Nord total,61147,88986


In [18]:
# first we create the sums per week
weekly_sums = data2betested.resample("W").sum()
# then we generate the weekly means for each quarter
quarterly_sums = data2betested.resample("Q").sum()
# for readability we'll revert the values back to integers
weekly_sums.dropna().astype(int).head(5)

,In,Out
Timestamp,,
2021-01-03,29814,25225
2021-01-10,133245,106920
2021-01-17,148006,125984
2021-01-24,136580,114336
2021-01-31,131532,106527


### Visualisierungen nach Zeitausschnitten

Liniendiagramm 
[Link zur Doku](https://altair-viz.github.io/gallery/multiline_highlight.html)

In [19]:
weekly_sums.dtypes
weekly_sums.loc["2021-10-04":"2021-12-06"]
#weekly_sums.info()

,In,Out
Timestamp,,
2021-10-10,226372,193536
2021-10-17,199191,161766
2021-10-24,212025,174148
2021-10-31,231163,199954
2021-11-07,220905,190681
2021-11-14,215369,185536
2021-11-21,216899,187518
2021-11-28,215294,184358
2021-12-05,208231,178654


In [20]:
test = weekly_sums.reset_index().melt("Timestamp")
test

,Timestamp,variable,value
0,2021-01-03,In,29814
1,2021-01-10,In,133245
2,2021-01-17,In,148006
3,2021-01-24,In,136580
4,2021-01-31,In,131532
...,...,...,...
101,2021-12-05,Out,178654
102,2021-12-12,Out,167535
103,2021-12-19,Out,170294
104,2021-12-26,Out,105752


In [21]:
alt.Chart(weekly_sums.reset_index().melt("Timestamp")).mark_circle(size=40).encode(
    x='Timestamp',
    y='value',
    color = 'variable',
    #color=alt.Color('value', scale=alt.Scale(scheme='blues')),
    tooltip=['variable','value']
).interactive().properties(width=800, height=400)

alt.Chart(...)

In [22]:
alt.Chart(weekly_sums.reset_index().melt("Timestamp")).mark_line(strokeWidth=1).encode(
    x='Timestamp',
    y='value',
    color='variable',
    tooltip=['variable','value']    
).properties(width=800, height=400).interactive()

alt.Chart(...)

In [23]:
# df_statzonen = df_statzonen[(df_statzonen.knr>0) & (df_statzonen.qname!="Hirslanden")]

highlight = alt.selection(type='single', on='mouseover', fields=['In'], nearest=True)
#x='date:StichtagDatJahr',
base = alt.Chart(weekly_sums).encode(
    x='Out',
    y='In',
    color='In'
)

points = base.mark_circle().encode(
    opacity=alt.value(1)
).add_selection(
    highlight
).properties(
    width=700 , height=300
)
lines = base.mark_line().encode(
    size=alt.condition(~highlight, alt.value(1.5), alt.value(4))
).interactive()

points + lines

alt.LayerChart(...)

---------------------------------------------------------- hier habe ich nicht mehr weiter gemacht

In [24]:
alt.Chart(mySelection).mark_circle(size=60).encode(
    x='StichtagDatJahr',
    y='AlterV20Kurz',
    #color='AnzNat',
    #color=alt.Color('AnzNat', scale=alt.Scale(scheme='dark2')),
    color=alt.Color('AlterV20Kurz', scale=alt.Scale(scheme='greens')),
    tooltip=['StichtagDatJahr','AlterV20Kurz','AnzBestWir']
).interactive() # this makes the axes interactive: now you can zoom & pan




NameError: name 'mySelection' is not defined

**Test: Choroplethenkarte**
Importiere die Geodaten als GeoJSON

Joine die importierten statistischen Daten des aktuellen Jahres zum Geodatensatz:

*Siehe dazu Doku zu [Geopandas](https://geopandas.org/docs/user_guide/mergingdata.html)*